In [ ]:
import xarray as xr
import datetime as dt
import pandas as pd
import numpy as np
adir_adcp = 'F:/data/cruise_data/saildrone/baja-2018/adcp_data/'
adir = 'F:/data/cruise_data/saildrone/baja-2018/'

filename_loggers_usv=adir + 'saildrone-gen_4-baja_2018-sd1002-20180411T180000-20180611T055959-1_minutes-v1_withloggers.nc'
filename_adcp = adir_adcp + 'saildrone-gen_4-baja_2018-sd1002-adcp-20180411T175730-20180611T235730_300s-v1.nc'

ds_usv = xr.open_dataset(filename_loggers_usv)  #1 minute averaging
ds_adcp = xr.open_dataset(filename_adcp)            #10 minute sampling
offset=0

In [ ]:
print(ds_adcp.time[0:10].data)
print(ds_usv.time[0,0:10].data)
print(type(ds_adcp.time))
ds_adcp

In [ ]:
#ds_usv2=ds_usv.sel(trajectory=1002)

#ds_usv2.time
#ds_usv2.coords('time',ds_usv2('time'))
#ds_temds.coords['day'] = ('time', [6, 7, 8])
#ds_usv2.assign_coords(obs=ds_usv2.time)
#ds_usv2

In [ ]:
tdim=ds_adcp.time[:,0].shape
tdim
print(ds_adcp.time)

In [ ]:
istart=0
print(tdim[0])
for i in range(0,tdim[0]):
    upper_time=ds_adcp.time[i,0]
    lower_time=upper_time-np.timedelta64(150,'s')
    #print(lower_time.data,upper_time.data)
    ds_usv2=ds_usv.where((ds_usv.time>lower_time) & (ds_usv.time<=upper_time)).mean() #,upper_time))
    if istart == 0:
        ds_usv3=ds_usv2
        istart=1
    else:
        ds_usv3=xr.concat([ds_usv3, ds_usv2], 'obs')

In [ ]:
ds_usv2
ds_usv3=xr.concat([ds_usv2, ds_usv2], 'obs')
ds_usv3
#xr.merge([ds_adcp, ds_usv2])

In [ ]:
pt = pd.to_datetime(ds_usv2.time.data)+offset  #add offset seconds
psec=pt.second
tem=ds_usv2 #[(psec <=12)]  #data average should be only use sec 54 to sec 06 of each minute 
tem['time']=tem['time']+offset #resample goes from :00 to :59 so shift +6sec
#print(tem.time)
xf_avg1 = tem.resample(time='10min').mean()
